In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

__Vasicek Model:__ $dr_t = \alpha(b-r_t)dt + \sigma dW_t$   
Точное решение: $r_t \sim N\left(r_0 e^{-\alpha t} + b (1-e^{-\alpha t}),\,\tfrac{\sigma^2}{2\alpha}\left(1-e^{-2\alpha t}\right)\right)
$

In [2]:
def minimizeNLogLikelihood(r):
    dt = 1 
    rt = r    
    N = r.shape[0]
    params0 = np.array([0.0000001,0.0000001,0.0000001])
    
    def VasicekLogLikelihood(params):
        alpha, b, sigma = params[0], params[1], params[2] 
        return -(N-1)*np.log(alpha/(sigma**2*(1-np.exp(-2*alpha*dt)))) \
            - alpha/(sigma**2*(1-np.exp(-2*alpha*dt)))*np.sum(-(rt[:-1] - rt[1:]*np.exp(-alpha*dt)-b*(1-np.exp(-alpha*dt)))**2) 
    
    params_opt = minimize(VasicekLogLikelihood, params0, method='Powell')
    return params_opt.x

In [3]:
def ClosestTradeDate(day, dates):
    return dates[dates <= day][-1]
    

In [4]:
def estimate_params(components, n_components):
    params = np.zeros((n_components,3))

    for i in range(n_components):
        params[i] = minimizeNLogLikelihood(components[:,i])

    return params

In [5]:
def load_bonds(bond_count, forDate):
 
    bond_info = {}

    for i in range( bond_count):
        bond_i = i

        coupons =pd.read_excel('офз_график_выплат.xlsx', bond_i)[['Date', 'Coupon']].astype({'Date':'M8[D]'})

        coupons_date = coupons['Date'].values


    
        info = pd.read_excel('офз_описание.xlsx',bond_i, index_col=0)
        num_coupons = info.loc['Периодичность выплаты купона в год'].values[0]
        coupons_rate = coupons['Coupon'].values/num_coupons/100
        coupons_rate = coupons_rate[coupons_date > forDate]

        coupons_date = coupons_date[coupons_date > forDate].reshape(-1,1)
        expDate = np.datetime64(info.loc['Дата погашения'].values[0])

        face = info.loc['Номинальная стоимость'].values[0]

        bond_info[i] = {
            'coupons_date': coupons_date,
            'coupons_rate': coupons_rate,
            'expDate': expDate,
            'face': face
        }
    return bond_info


In [6]:
# def VasicekPath(params,r0, N):
#     dt = 1    
#     alpha, b, sigma = params[0], params[1], params[2]
#     r = [] # варианты аля-ставки на один из аля-TtM  
#     for j in range(N):
#         r1 = r0 + alpha*(b-r0)*dt + sigma*np.sqrt(dt)*np.random.randn()
#         r.append(r1)
#     return r

def VasicekPath(params,r0, N, days_for_predict):
    dt = 1    
    alpha, b, sigma = params[0], params[1], params[2]
    simulations = np.zeros((N,days_for_predict)) # варианты аля-ставки на один из аля-TtM  
    
    
    for j in range(N):
        r = [r0]
        for d in range(days_for_predict):
            R = alpha*(b-r[-1])*dt + sigma*np.sqrt(dt)*np.random.randn()
            r.append(r[-1] + R)
        simulations[j] = r[1:]
    return simulations

In [7]:
from scipy.interpolate import interp1d


def interpolate(bond_count, pca, r0, bond_info, N, days_for_predict):
    inverse_r0 = np.array([r0.values for i in range(N)])
    
    x_axis = [0]+list(r0.index*360)
    xnew = np.arange(0, 30*360, 1)
    
    for bond_i in range(bond_count):
    #переводим аля-ставки в разности реальных ставок 
        inverse_r = pca.inverse_transform(bond_info[bond_i]['simulations'])
        ytm = np.cumsum(np.concatenate((inverse_r0[None, :, :], inverse_r)), 0)

        inverse_r_interpolate = np.zeros((days_for_predict+1, N, len(xnew)))

        for i, data in enumerate(ytm):
            for k, ofz in enumerate(data):
                y = [0]+list(ofz)

                f = interp1d(x_axis, y)
                ynew = f(xnew)

                inverse_r_interpolate[i, k] = ynew
                
        bond_info[bond_i]['inverse_r_interpolate'] = inverse_r_interpolate
    return bond_info

In [8]:
def PriceBond(bond_params, tradeDate, days_for_predict,  N=10):
    coupons_date = bond_params['coupons_date']
    couponRate = bond_params['coupons_rate']
    FaseValue = bond_params['face']/10
    inverse_r_interpolate = bond_params['inverse_r_interpolate']

    TimeToMaturity = np.apply_along_axis(lambda x: int((x - tradeDate)/np.timedelta64(1,'D')), 1, coupons_date)
    B = np.zeros((N, days_for_predict+1))
#     discount_factor = ((1+inverse_r_interpolate[:, :TimeToMaturity[-1]])**np.arange(TimeToMaturity[-1]))[:, ::-1]


#     for i in range(N):    
#         #для каждого купона считаем его динамику цены (делим значение купона на показатель дисконтирования)
#         for j,_ in enumerate(TimeToMaturity):        
#             V = FaseValue*couponRate[j] if j != len(TimeToMaturity)-1 else FaseValue*(1+couponRate[j])  
#             b = (V / discount_factor[i, :TimeToMaturity[j]])  
#             B[i, :TimeToMaturity[j]] += b

    for i in range(N): 
        
        #для каждого купона считаем его динамику цены (делим значение купона на показатель дисконтирования)
        for j,k in enumerate(TimeToMaturity):        
            V = FaseValue*couponRate[j] if j != len(TimeToMaturity)-1 else FaseValue*(1+couponRate[j])  

            X = []
            for d in range(days_for_predict+1):

                X.append(V/(1+inverse_r_interpolate[d][i][k-d])**(k-d))

            B[i] += X
        
    return B

In [9]:
def estimate_r(bond_count, params, r0_diff, days_for_predict=1,  N=100):
    for bond_i in range(bond_count):
        r = []    
        for j in range(params.shape[0]):        
            r.append(VasicekPath(params[j], r0_diff[j], N, days_for_predict))

        bond_info[bond_i]['simulations'] = np.array(r).T
    return bond_info

In [10]:
def estimate_price(bond_count, bond_info, N, tradeDate):

    for i in range(bond_count):

        p = PriceBond(bond_info[i], tradeDate,  days_for_predict, N=N)

        bond_info[i]['price_path'] = p
        
        
    return bond_info

### смотрим на весь портфель из облигаций на 1 день

In [11]:
def load_real_price(bond_count, tradeDate, forecastDate):
    p0 = np.array([])
    p_last = np.array([])

    for i in range(bond_count):
        bond_price = pd.read_excel('офз_данные.xlsx', i, index_col=2).iloc[:, 6:7]
        bond_price.columns=['price']


        p0 = np.append(p0, bond_price.loc[tradeDate]['price'])
        p_last = np.append(p_last, bond_price.loc[forecastDate]['price'])

#         bond_result = bond_info[i]
#         dates_index = pd.date_range(tradeDate1, bond_result['expDate'])[:-1]
#         p_path = pd.DataFrame(bond_result['price_path'], columns=dates_index)
        


    return p0, p_last

In [12]:
def pricePortfolioInitial(p_real, bond_value = 1):
    w = bond_value/p_real
    return np.sum(w*p_real), w

def pricePortfolio(price, w):
    if len(price.shape) == 2:
        return np.sum(w[:, None]*price, 0)
    else: 
        return np.sum(w*price)

In [13]:
def rebalance(amount0, p_path):
    w = amount0 / sum(amount0)
    
    count = amount0 / p_path[:, 0]
    
    for i in range(1, p_path.shape[1]):
        p = p_path[:, i]
        port_value = sum(p * count)
        count = (port_value * w) / p
        
        
    return sum(count * p)


def estimate_VaR(amount0, p_pathes, N, alpha=0.99):
    p0_real = p_pathes[:, 0, 0]
    
    port_value_0 = sum(amount0)
    
    port_value_last = np.zeros(N)
    
    for i in range(N):
        p_path = np.array([x[i] for x in p_pathes])
        port_value_last[i] = rebalance(amount0, p_path)
        
    distr = (port_value_0 - port_value_last) / port_value_0

    return np.percentile(distr, alpha*100), distr
    
def real_loss(p0, p_last, amount0):
    count = amount0 / p0
    return((sum(amount0) - sum(count * p_last)) / sum(amount0))
    

In [14]:
def make_true_price_path(bond_count, bond_info, p0_real, N, days_for_predict):
    
    
    for i in range(bond_count):
        yields = bond_info[i]['price_path'][:, 1:] / bond_info[i]['price_path'][:, :-1]
        yields = yields[:, :days_for_predict]

        bond_info[i]['price_path'] = np.cumprod(np.concatenate((p0_real[i] + np.zeros((N,1)), 
                   yields), 1), 1)
        
        
    return  bond_info    

In [24]:
ir = pd.read_excel('офз.xlsx', index_col=0, header=1) / 100/365

dates = np.array(['-'.join(x.split('.')[::-1]) for x in ir.index]).astype('M8[D]')
ir.index = dates
ir.sort_index(inplace=True)
dates = dates[::-1][1:]

ir_diff = ir.diff().dropna()# берем разности

n_components = 3
pca = PCA(n_components=n_components)
components = pca.fit_transform(ir_diff)




In [25]:
result = []
startDate0 = np.datetime64('2015-01-12')
days_for_predict = 1
bond_count = 5
N = 100
    
#result = pd.DataFrame(columns=['forDate', 'VaR', 'loss'])

for forecastDate in dates[dates >= np.datetime64('2018-01-12')][:]:
    
    startDate = startDate0 + (forecastDate - np.datetime64('2018-01-12'))
    
    tradeDate =  ClosestTradeDate(forecastDate-1, dates)
    
    print('last traid date before forecase', tradeDate)
    print('forecast date', forecastDate)
    bond_info = load_bonds(bond_count, tradeDate)
    
    components_for = components[(dates <= tradeDate) & (dates >= startDate)]
    params = estimate_params(components_for, n_components)
    
    r0_diff = components_for[-1]
    r0 = ir.loc[tradeDate]
    
    bond_info = estimate_r(bond_count, params,  r0_diff, days_for_predict=days_for_predict, N=N)
    bond_info = interpolate(bond_count, pca, r0, bond_info, N, days_for_predict)
    bond_info = estimate_price(bond_count, bond_info, N, tradeDate)
    
    p0, p_last = load_real_price(bond_count, tradeDate, forecastDate)

    bond_info = make_true_price_path(bond_count, bond_info, p0, N, days_for_predict)
    
    porfolio_path = np.array([bond_info[i]['price_path'] for i in range(bond_count)])

    amount0 = np.array([10e6]*5)
    VaR, distr = estimate_VaR(amount0, porfolio_path, N)
    
    loss = real_loss(p0, p_last, amount0 )
    
    print('VaR', VaR, 'loss', loss)
    print()
    result.append([forecastDate, VaR, loss])

last traid date before forecase 2018-01-11
forecast date 2018-01-12
VaR 0.007375328424706357 loss -0.0023742910913793743

last traid date before forecase 2018-01-12
forecast date 2018-01-15
VaR 0.01859456179896934 loss -0.00039775093778461216

last traid date before forecase 2018-01-15
forecast date 2018-01-16
VaR 0.012154671523574275 loss 0.0005815511576956511

last traid date before forecase 2018-01-16
forecast date 2018-01-17
VaR 0.002479807419562136 loss 0.0005381177560760081

last traid date before forecase 2018-01-17
forecast date 2018-01-18
VaR 0.008952180157946176 loss -0.00026176909669011834

last traid date before forecase 2018-01-18
forecast date 2018-01-19
VaR 0.004623377158583121 loss -0.0007562153502005339

last traid date before forecase 2018-01-19
forecast date 2018-01-22
VaR 0.007082657429296316 loss 0.0004233328567011654

last traid date before forecase 2018-01-22
forecast date 2018-01-23
VaR 0.005268676825906028 loss 1.18642191067338e-05

last traid date before forec

VaR 0.008454680617108903 loss -0.0009481460944987833

last traid date before forecase 2018-04-20
forecast date 2018-04-23
VaR 0.0033121107341520586 loss 0.001764808611332327

last traid date before forecase 2018-04-23
forecast date 2018-04-24
VaR 0.0058465461193085095 loss -0.002683173467960209

last traid date before forecase 2018-04-24
forecast date 2018-04-25
VaR 0.008100127880107649 loss 0.0015061108100901544

last traid date before forecase 2018-04-25
forecast date 2018-04-26
VaR 0.0037877007495341667 loss 0.0011700997565197945

last traid date before forecase 2018-04-26
forecast date 2018-04-27
VaR 0.004732795676531102 loss -0.0030104140252541003

last traid date before forecase 2018-04-27
forecast date 2018-04-28
VaR 0.009358007655495578 loss -0.00034673623461693525

last traid date before forecase 2018-04-28
forecast date 2018-05-03
VaR 0.007609696715931599 loss 0.004044265498311221

last traid date before forecase 2018-05-03
forecast date 2018-05-04
VaR 0.0028343915158136274 l

VaR 0.002509015487164705 loss -0.0008887593771879375

last traid date before forecase 2018-07-30
forecast date 2018-07-31
VaR 0.007799352010144359 loss -0.0015820793180656434

last traid date before forecase 2018-07-31
forecast date 2018-08-01
VaR 0.008148923638259649 loss 0.001281305745716393

last traid date before forecase 2018-08-01
forecast date 2018-08-02
VaR 0.00111747947745814 loss 0.0035465219539541006

last traid date before forecase 2018-08-02
forecast date 2018-08-03
VaR -0.004027713238415947 loss 0.0006483837670338154

last traid date before forecase 2018-08-03
forecast date 2018-08-06
VaR 0.005701538867652462 loss 0.0017396238654358684

last traid date before forecase 2018-08-06
forecast date 2018-08-07
VaR 0.00014776198394507835 loss -0.0005046663092574477

last traid date before forecase 2018-08-07
forecast date 2018-08-08
VaR 0.0036637810721913654 loss 0.008266136486375034

last traid date before forecase 2018-08-08
forecast date 2018-08-09
VaR -0.004871759473569867 lo

VaR 0.004303388029531414 loss 0.0010856413406065107

last traid date before forecase 2018-11-01
forecast date 2018-11-02
VaR 0.0033950139194888988 loss -0.0005645538382591307

last traid date before forecase 2018-11-02
forecast date 2018-11-06
VaR 0.0043910987866321045 loss 0.0002640659557203949

last traid date before forecase 2018-11-06
forecast date 2018-11-07
VaR 0.00209920374968001 loss 0.001991366255887449

last traid date before forecase 2018-11-07
forecast date 2018-11-08
VaR 0.000305475038308317 loss 0.004866385828157514

last traid date before forecase 2018-11-08
forecast date 2018-11-09
VaR -0.004051050454858943 loss 0.003019277396444678

last traid date before forecase 2018-11-09
forecast date 2018-11-12
VaR -0.0019538986697151864 loss 0.001577959273917377

last traid date before forecase 2018-11-12
forecast date 2018-11-13
VaR 0.0013719966884677734 loss 0.0011377262110257148

last traid date before forecase 2018-11-13
forecast date 2018-11-14
VaR -0.0005730508605475612 los

In [26]:
result = pd.DataFrame(result, columns=['forDate', 'VaR', 'loss'])


In [27]:
sum(result['loss'] > result['VaR'])/result.shape[0]

0.13877551020408163